 ### Essential Libraries

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer #for stemming of words
stemmer=LancasterStemmer

import tensorflow
import random
import json
import numpy
import tflearn
import pickle

C:\Users\Hamza-PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hamza-PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hamza-PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hamza-PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWa

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


### Loading data from json file

In [2]:
#loading json file through library
with open("c++_assignment.json") as file:
    data = json.load(file)
#we'll be training our model on this data

### Tokenizing data for further use

In [3]:
#creating empty lists to store tokenized data
words=[]
labels=[]
docs_x=[]
docs_y=[]

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        #tokenizing each word in pattern
        wrds= nltk.word_tokenize(pattern)
        #adding words to words list
        words.extend(wrds)
        
        docs_x.append(wrds)
       # print(docs_x)
        docs_y.append(intent["tag"])
       # print(docs_y)
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
        

### Stemming & Removing duplicates

In [4]:
obj=stemmer()
words = [obj.stem(w.lower()) for w in words if w!="?"]
words = sorted(list(set(words)))
labels = sorted(labels)

### One-Hot Encoding

In [5]:
training = []
output = []

out_empty=[0 for _ in range(len(labels))]

#x=0,1,2... doc= words
for x,doc in enumerate(docs_x):
    bag = []
    
    wrds = [obj.stem(w) for w in doc]
    
    
    for w in words:
        if w in wrds: 
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])]=1
    training.append(bag)
    output.append(output_row)

   

    
        

### Converting data to numpy arrays for tflearn

In [6]:
training = numpy.array(training)
output=numpy.array(output)  


### Setting up our neural network

In [7]:
tensorflow.reset_default_graph() #Reset Underlying Graph data

#Building our own Neural Network
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

#Defining Model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

#Now we have setup model, now we need to train that model by fitting data into it by model.fit()
#n_epoch is the number of times that model will se our data during training

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True) 
model.save("model.tflearn") #Saving the model

Training Step: 5999  | total loss: 1.43330 | time: 0.012s
| Adam | epoch: 1000 | loss: 1.43330 - acc: 0.9094 -- iter: 40/48
Training Step: 6000  | total loss: 1.29384 | time: 0.014s
| Adam | epoch: 1000 | loss: 1.29384 - acc: 0.9185 -- iter: 48/48
--
INFO:tensorflow:C:\Users\Hamza-PC\Desktop\Chatbot_AI\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### prediction of user input

In [8]:
#Cleaning User Input
def bag_of_words(sentence,words):
    
    bag = [0 for _ in range(len(words))]
    
    # Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence) #Again tokenizing the sentence
    
    #Stemming each word from the user's input
    sentence_words= [obj.stem(word.lower()) for word in sentence_words]

    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i]=1
    
    return numpy.array(bag)

def chat():
    print("Start talking to bot and get your queries resolved!(Enter 'quit' to STOP CHATTING)")
    while True:
        inputt=input("You: ")
        if inputt.lower()=="quit":
            break
        
        results = model.predict([bag_of_words(inputt,words)])[0]
        results_index=numpy.argmax(results)
        
        tag = labels[results_index]
        
        if results[results_index] > 0.5:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                
            print(random.choice(responses))
        else:
            print("Sorry i didn't get that. Try again! ")
             
        
                
                 
    

### Main

In [9]:
chat()

Start talking to bot and get your queries resolved!(Enter 'quit' to STOP CHATTING)
You: hi
Hi there, how can I help?
You: your name?
I'm hamza!
You: what is your age?
21 years young!
You: why were you made?
i was made to help you in c++ assignments
You: how to write my first c++ code?
https://www.geeksforgeeks.org/writing-first-c-program-hello-world-example/
You: how to store data?
https://www.geeksforgeeks.org/data-structures/
You: how to play with pointers?
https://www.geeksforgeeks.org/pointers-in-c-and-c-set-1-introduction-arithmetic-and-array/
You: how to write conditional statements?
https://www.geeksforgeeks.org/decision-making-c-c-else-nested-else/
You: ddadsda
Hello!
You: quit
